# 수열과 쿼리 15

## 문제

* 길이가 N인 수열 A1, A2, ..., AN이 주어진다. 이때, 다음 쿼리를 수행하는 프로그램을 작성하시오.
    * 1 i v : Ai를 v로 바꾼다. (1 ≤ i ≤ N, 1 ≤ v ≤ 109)
    * 2 : 수열에서 크기가 가장 작은 값의 인덱스를 출력한다. 그러한 값이 여러개인 경우에는 인덱스가 작은 것을 출력한다

* 수열의 인덱스는 1부터 시작한다.

## 입력

* 첫째 줄에 수열의 크기 N이 주어진다. (1 ≤ N ≤ 100,000)

* 둘째 줄에는 A1, A2, ..., AN이 주어진다. (1 ≤ Ai ≤ 109)

* 셋째 줄에는 쿼리의 개수 M이 주어진다. (1 ≤ M ≤ 100,000)

* 넷째 줄부터 M개의 줄에는 쿼리가 주어진다.

## 출력

* 2번 쿼리에 대해서 정답을 한 줄에 하나씩 순서대로 출력한다.

## Code

In [1]:
#14427_수열과 쿼리 15

import sys
import heapq

submit = False
input = sys.stdin.readline if submit else input

def solution():
    N = int(input())
    arr = list(map(int, input().split()))
    arr = [[v, i] for i, v in enumerate(arr)]
    heapq.heapify(arr)

    stack, answer = [], []
    M = int(input())
    for _ in range(M):
        query = list(map(int, input().split()))
        cmd = query[0]
        if cmd == 1:
            i, v = query[1:]
            while True:
                temp = heapq.heappop(arr)
                if temp[1] == (i-1):
                    temp[0] = v
                    heapq.heappush(arr, temp)
                    while stack:
                        heapq.heappush(arr, stack.pop())
                    break
                else:
                    stack.append(temp)

        elif cmd == 2:
            answer.append(arr[0])
            
    print(*[i+1 for _, i in answer], sep='\n')

In [ ]:
#14427_수열과 쿼리 15

import sys
import math

submit = False
input = sys.stdin.readline if submit else input

class SegmentTree:
    def __init__(self, arr):
        self.arr = arr
        h = math.ceil(math.log2(len(arr))) + 1
        nodeNum = 1 << h
        # nodeNum = 1 ** h
        self.seg = [0 for _ in range(nodeNum)]
        self.makeSegTree(self.arr, self.seg, 1, 0, len(arr) - 1)

    def makeSegTree(self, arr, seg, idx, st, ed):
        if st == ed:
            seg[idx] = arr[st]
            return seg[idx]
        
        mid = (st + ed) // 2
        left, right = self.makeSegTree(arr, seg, idx * 2, st, mid), self.makeSegTree(arr, seg, idx * 2 + 1, mid+1, ed)
        seg[idx] = min(left, right)
        return seg[idx]

    def find(self, range1, range2, idx, st, ed):
        if range2 < st or range1 > ed:
            return [10**9+1, 10**9+1]
        if range1 <= st and ed <= range2:
            return self.seg[idx]
        
        mid = (st + ed) // 2
        left, right = self.find(range1, range2, idx * 2, st, mid), self.find(range1, range2, idx * 2 + 1, mid + 1, ed)
        return min(left, right)
   
    def update(self, update_idx, update_num, idx, st, ed):
        if (st > update_idx) or (ed < update_idx):
            return 

        if st == ed:
            self.seg[idx][0] = update_num
            return

        mid = (st + ed) // 2
        self.update(update_idx, update_num, idx * 2, st, mid)
        self.update(update_idx, update_num, idx * 2 + 1, mid + 1, ed)
        self.seg[idx] = min(self.seg[idx*2], self.seg[idx*2+1])

def solution():
    N = int(input())
    arr = list(map(int, input().split()))
    arr = [[v, i] for i, v in enumerate(arr)]
    segtree = SegmentTree(arr)

    answer = []
    M = int(input())
    for _ in range(M):
        query = list(map(int, input().split()))
        cmd = query[0]
        if cmd == 1:
            i, v = query[1:]
            segtree.arr[i-1] = [v, i-1]
            segtree.update(i, v, 1, 1, N)
        elif cmd == 2:
            answer.append(segtree.find(1, N, 1, 1, N))

    print(*[i+1 for _, i in answer], sep='\n')

## 예제입력 - 출력

In [2]:
solution()

5
4
3


## Note

### 풀이 1

* 1. 우선순위 큐에 [값, index]의 List를 저장하여 항상 최소값이 0번 index에 위치하게 한다
* 2. 값을 갱신할 i, v가 주어지면 우선순위 큐에서 index 값이 i인 값이 나타날 때까지 저장된 값을 추출 후 값을 갱신한다.
    * 1\) 추출한 List의 index가 (i-1)이 아닌 경우는 다시 저장해야 하므로 List(stack)에 임시로 저장한다.
    * 2\) 추출한 List의 index가 (i-1)와 동일하면 값을 변경 후 다시 우선순위 큐에 저장한다.
    * 3\) stack에 임시로 저장했던 모든 List를 다시 우선순위 큐에 저장한다.
* 3. 최소값은 항상 우선순위 큐의 0번 index에 위치하므로 최소값을 찾아야 하면 0번 index의 값을 List(answer)에 저장한댜.
* 4. answer에서 index의 값에 1을 더한 값을 출력한다.

### 풀이 2

* 1. Segment Tree를 이용하여 구간 별 최소값을 미리 저장한 후 원하는 구간의 최소값을 불러온다.
    * 1\) makeSegTree를 통해 입력받은 숫자의 구간 별 최소 값을 미리 저장한다.
    * 2\) 전체 구간을 반으로 나눠 탐색하는 구간의 시작 index와 끝 index가 같을 때 까지 구간을 나눈다.
    * 3\) 구간의 시작 index와 끝 index가 같으면 해당 Node의 최소값은 자기 자신이므로 값을 List(seg)에 저장한다.
    * 4\) 최하단 Node의 상단 Node는 숫자 2개 구간의 최소, 최대값이 저장되게 되고 반복되면서 반으로 나눴던 모든 구간의 최소, 최대값이 저장된다.
        * 저장할 때, 상위 index의 *2와 *2+1에 저장한다.
        * 1 -> 2, 3 / 2 -> 4, 5 / 3 -> 6, 7 ...
        * eg. [5, 4, 3, 2, 1]
        * index: 1 -> 2, 3 / 2 -> 4, 5 / 3 -> 6, 7 ...
            * index 1: 전체 구간의 최소값 (1, 4)
            * # ------------------------------- # 
            * index 2: 1(5) ~ 3(3) 구간의 최소값 (3, 2)
            * index 3: 4(2) ~ 5(1) 구간의 최소값 (1, 4)
            * # ------------------------------- #
            * index 4: 1(5) ~ 2(4) 구간의 최소값 (4, 1)
            * index 5: 3(2) ~ 3(2) 구간의 최소값 (2, 3)
            * index 6: 3(2) ~ 3(2) 구간의 최소값 (2, 3)
            * index 7: 4(1) ~ 4(1) 구간의 최소값 (4, 1)
            * # ------------------------------- #
            * index 8: 1(5) ~ 1(5) 구간의 최소값 (5, 0)
            * index 9: 2(4) ~ 2(4) 구간의 최소값 (4, 1)

* 2. 숫자의 변동에 따라 Segment Tree를 업데이트 한다.
    * 1\) 재귀 함수를 사용하여 전체 index를 탐색하여 숫자를 바꿀 index를 찾는다
        * 탐색한 구간의 시작 index와 끝 index가 숫자를 바꿀 index와 같아질때까지의 구간을 찾는다.
    * 2\) 재귀 함수를 통해 특정 index의 값을 업데이트 하고 나면 segment tree에서 상위 구간의 최소값을 갱신한다.
    * eg. 
    * numbers: [5, 4, 3, 2, 1]
    * seg: [0, [1, 4], [3, 2], [1, 4], [4, 1], [3, 2], [2, 3], [1, 4], [5, 0], [4, 1], 0, 0, 0, 0, 0, 0]
    * # ------------------------------- #
    * numbers: [5, 4, 3, 2, 3]
    * seg: [0, [2, 3], [3, 2], [2, 3], [4, 1], [3, 2], [2, 3], [3, 4], [5, 0], [4, 1], 0, 0, 0, 0, 0, 0]

* 3. Segment Tree를 탐색하면서 전체 구간의 최소값을 구한다.
    * 1\) 최소값을 찾을 구간을 전체 구간으로 주어 전체 구간의 최소값을 찾는다
    * 2\) index + 1값을 출력한다.


https://www.acmicpc.net/problem/14427